# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, make_scorer

from tqdm import tqdm
from time import time

In [2]:
X_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')

X_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

X_val = np.load('data/x_val.npy')
y_val = np.load('data/y_val.npy')

In [3]:
X_train = np.concatenate((X_train, X_val), axis=0)
y_train = np.concatenate((y_train, y_val), axis=0)

# Playground

In [4]:
np.unique(y_train, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18]),
 array([ 37592,   2279,   1674,  18202,  17706,     23,   4058,  30357,
         16645,     90,   4762, 324955,  61707,    812,  25292,   1693,
           367,    916,  11789]))

11 is normal class

In [5]:
normal_data = X_train[y_train == 11] # 11 is normal class

In [23]:
len(normal_data)

324955

# Helpers

In [6]:
def evaluate_model(model, y_test, y_pred):
    positive_class = 'normal'
    
    accuracy = accuracy_score(y_test, y_pred)
    
    precision = precision_score(y_test, y_pred, pos_label=positive_class)
    recall = recall_score(y_test, y_pred, pos_label=positive_class)
    f1 = f1_score(y_test, y_pred, pos_label=positive_class)
    
    metrics = {
        'model': [model],
        'accuracy': [accuracy],
        'precision': [precision],
        'recall': [recall],
        'f1': [f1],
    }
    
    return metrics

# SGD-OCSVM

In [7]:
from sklearn.linear_model import SGDOneClassSVM

In [183]:
sgd_ocsvm_start = time()

sgd_ocsvm = SGDOneClassSVM(nu=0.02, max_iter=1000, tol=0.001, learning_rate='constant', eta0=0.06, random_state=42)
sgd_ocsvm.fit(normal_data)
        
print(f"seconds: {time() - sgd_ocsvm_start}")

seconds: 0.4351677894592285


In [184]:
y_pred_test_sgd_ocsvm = sgd_ocsvm.predict(X_test)
y_pred_test_sgd_ocsvm = np.where(y_pred_test_sgd_ocsvm == 1, 'normal', 'anomaly')
y_test_converted_sgd_ocsvm = np.where(y_test == 11, 'normal', 'anomaly')

In [185]:
sgd_ocsvm_df = pd.DataFrame(evaluate_model("sgd_ocsvm", y_test_converted_sgd_ocsvm, y_pred_test_sgd_ocsvm))
sgd_ocsvm_df

,model,accuracy,precision,recall,f1
0,sgd_ocsvm,0.660094,0.631882,0.990054,0.77142


# PCA Reconstruction

In [186]:
from sklearn.decomposition import PCA

In [240]:
pca_start = time()

pca = PCA(n_components=2, whiten=True, svd_solver='auto')

normal_data_pca = pca.fit_transform(normal_data)
normal_data_reconstructed = pca.inverse_transform(normal_data_pca)

reconstruction_error = np.mean((normal_data - normal_data_reconstructed) ** 2, axis=1)

threshold = np.percentile(reconstruction_error, 95)

X_test_pca = pca.transform(X_test)
X_test_reconstructed = pca.inverse_transform(X_test_pca)
X_test_reconstruction_error = np.mean((X_test - X_test_reconstructed) ** 2, axis=1)

print(f"seconds: {time() - pca_start}")

seconds: 3.2669599056243896


In [241]:
y_pred_test_pca = np.where(X_test_reconstruction_error > threshold, 'anomaly', 'normal')
y_test_converted_pca = np.where(y_test == 11, 'normal', 'anomaly')

In [242]:
pca_df = pd.DataFrame(evaluate_model('PCA Reconstruction', y_test_converted_pca, y_pred_test_pca))
pca_df

,model,accuracy,precision,recall,f1
0,PCA Reconstruction,0.872866,0.848401,0.950369,0.896494


# Results (2)

In [243]:
results_df = pd.concat([sgd_ocsvm_df, pca_df], ignore_index=True)
results_df.to_csv('results/results_xiiot_2.csv', index=False)

results_df

,model,accuracy,precision,recall,f1
0,sgd_ocsvm,0.660094,0.631882,0.990054,0.771420
1,PCA Reconstruction,0.872866,0.848401,0.950369,0.896494


# OCSVM

In [ ]:
from sklearn.svm import OneClassSVM

In [ ]:
ocsvm = OneClassSVM(kernel='rbf', gamma=5, nu=0.1)

batch_size = 50000
n_batches = 1# int(np.ceil(len(normal_data) / batch_size))

for i in tqdm(range(n_batches), desc="Training OCSVM"):
    start = i * batch_size
    end = min((i + 1) * batch_size, len(normal_data))
    batch_data = normal_data[start:end]
    
    if i == 0:
        ocsvm.fit(batch_data)
    else:
        ocsvm.fit(np.vstack((ocsvm.support_vectors_, batch_data)))

In [ ]:
y_pred_test_ocsvm = ocsvm.predict(X_test)
y_pred_test_ocsvm = np.where(y_pred_test_ocsvm == 1, 'normal', 'anomaly')
y_test_converted_ocsvm = np.where(y_test == 11, 'normal', 'anomaly')

In [ ]:
ocsvm_df = pd.DataFrame(evaluate_model("ocsvm", y_test_converted_ocsvm, y_pred_test_ocsvm))
ocsvm_df

,model,accuracy,precision,recall,f1
0,ocsvm,0.905448,0.93814,0.895863,0.916514


# LOF

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
lof_start = time()

lof = LocalOutlierFactor(n_neighbors=50, contamination=0.1, novelty=True)
lof.fit(X_train)

print(f"seconds: {time() - lof_start}")

seconds: 201.64672589302063


In [ ]:
y_pred_test_lof = lof.predict(X_test)
y_pred_test_lof = np.where(y_pred_test_lof == 1, 'normal', 'anomaly')
y_test_converted_lof = np.where(y_test == 11, 'normal', 'anomaly')

In [ ]:
lof_df = pd.DataFrame(evaluate_model("lof", y_test_converted_lof, y_pred_test_lof)) 
lof_df

,model,accuracy,precision,recall,f1
0,lof,0.617714,0.609559,0.946171,0.741449


# IF

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
if_start = time()

iforest = IsolationForest(n_estimators=200, max_samples=512, contamination=0.5, random_state=42)
iforest.fit(X_train)

print(f"seconds: {time() - if_start}")

seconds: 45.52945113182068


In [ ]:
y_pred_test_if = iforest.predict(X_test)
y_pred_test_if = np.where(y_pred_test_if == 1, 'normal', 'anomaly')
y_test_converted_if = np.where(y_test == 11, 'normal', 'anomaly')

In [ ]:
if_df = pd.DataFrame(evaluate_model("if", y_test_converted_if, y_pred_test_if)) 
if_df

,model,accuracy,precision,recall,f1
0,if,0.698781,0.778092,0.671586,0.720927


# Results

In [ ]:
results_df = pd.concat([ocsvm_df, lof_df, if_df], ignore_index=True)
results_df.to_csv('results/results_xiiot.csv', index=False)

results_df

,model,accuracy,precision,recall,f1
0,ocsvm,0.905448,0.938140,0.895863,0.916514
1,lof,0.617714,0.609559,0.946171,0.741449
2,if,0.698781,0.778092,0.671586,0.720927
